# First steps with SYGMA

调用函数

In [4]:
from NuPyCEE import sygma as s
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad
import os
import sys

## 1. The initial mass function

直接计算产出与模拟结果的对比（基于Salpeter IMF，设置恒星质量范围为 [1,30] ，形成的恒星总质量为1e11,每个恒星的产出为0.1 Msun ）

In [1]:
k_N=1e11*0.35/(1**(-0.35)-30**(-0.35))
N_tot=k_N*(1**(-1.35)-30**(-1.35))/1.35
Yield_tot=0.1*N_tot
print(Yield_tot/1e11)

0.03687728129719034


通过模拟计算上述产出

In [24]:
s1=s.sygma(iolevel=0,mgal=1e11,dt=1e7,imf_type='salpeter',imf_bdys=[1,30],iniZ=0.02,hardsetZ=0.02,
           table='yield_tables/agb_and_massive_stars_h1.txt',sn1a_on=False, sn1a_table='yield_tables/sn1a_h1.txt', 
           iniabu_table='yield_tables/iniabu/iniab_h1.ppn',pop3_table='yield_tables/popIII_h1.txt')
Yield_tot_sim=s1.history.ism_iso_yield[-1][0]
print(Yield_tot_sim)

SYGMA run in progress..
   SYGMA run completed - Run time: 0.02s
3687728129.719033


使用原文件报错：
Error-Model(M=12.0,Z=0.02) not found in myposition/NuPyCEE/yield_tables/agb_and_massive_stars_h1.txt
Error-Model(M=12.0,Z=0.01) not found in myposition/NuPyCEE/yield_tables/agb_and_massive_stars_h1.txt
我找到了这个文件并在其中加入了M=12,Z=0.02和M=12,Z=0.01，（后面表格内容复制了M=7.0的内容，应该需要查询一下正确数值）不再报错，且结果与正确结果只差0.00001.
猜测可能是某次更新时不小心删去了这一段。